<a href="https://colab.research.google.com/github/ciwwwnd/nlp/blob/main/hw2_4%D0%BA%D1%83%D1%80%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gzip
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import string 
nltk.download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
!wget http://snap.stanford.edu/data/amazon/Furniture.txt.gz

def parse_amazon_files(amazon_data):
  file_name = gzip.open(amazon_data, 'r')
  d = {}
  for line in file_name:
    line = line.strip().decode('utf-8')
    col = line.find(':')
    if col == -1:
      yield d
      d = {}
      continue
    product_name = line[:col]
    other = line[col + 2:]
    d[product_name] = other
  yield d

def get_df(amazon_data):
    data = []
    for item in parse_amazon_files(amazon_data):
        data.append(item)
    return pd.DataFrame(data)


--2022-12-17 19:53:35--  http://snap.stanford.edu/data/amazon/Furniture.txt.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20958 (20K) [application/x-gzip]
Saving to: ‘Furniture.txt.gz.10’

Furniture.txt.gz.10 100%[===================>]  20.47K  --.-KB/s    in 0.02s   

2022-12-17 19:53:35 (1.09 MB/s) - ‘Furniture.txt.gz.10’ saved [20958/20958]



In [3]:
df = get_df("/content/Furniture.txt.gz")
df.dropna(inplace=True)
df.sample(5)

,product/productId,product/title,product/price,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
66,B000PDN82O,Prepac Large Deluxe Storage Unit,271.36,AI8X3NNKB29SB,Karen Terry,0/0,4.0,1358035200,"Good product, but could be a little better","We have a similar, smaller storage cabinet fro..."
53,B000KRVZ84,Amazon.com: Lancaster Wardrobe Valet in Mahoga...,unknown,A1TGH5CPRXJ38K,Kevin L. Coleman,0/0,4.0,1359676800,It does the job,This was my first purchase of a wardrobe valet...
19,B000KU5D0M,Kitchen Cart with Granite Top Insert,unknown,A2VQYJ4PJ0PRHL,Thu A. Trang,1/1,5.0,1222732800,GREAT BUY!,The product is very well made. Love the granit...
38,B000MQKFBG,Lavender Wood Wick Candle 22 oz.,unknown,A1TC70XIHZJ5RU,"Catherine A. Hukill ""Cardscat68""",0/0,4.0,1231459200,quick delivery,Candle arrived quickly and in good shape.I gav...
82,B000HSCBSY,Smith &#38; Hawken&#174; Marlton 2007 Loveseat,unknown,A2PSLAL3NOLMBB,"T's Mom ""T-mom""",0/0,4.0,1226966400,Marlton 2007 Loveseat,"Easy to put together, very stylish. Cushions h..."


Способы извлекания упоминания товаров. 

1. Контекстные эмбеддинги -> по идее, слова, которые будут относится к одному и тому же товару (сущности), векторы будут рядом и расстояние между ними будет меньше, чем между любым рандомным. Соответственно, можно учитывать векторную близость между эмбеддингом всего текста и вектором названия предмета мебели, потому что предметы мебели не обладают чрезвычайно уникальными названиями (но к сожалению лексического мусора будет много, типа местоимения, глаголы, наречия, прилагательные, а не только искомая сущность)

2. Хардкодинг -> так как названия товаров конкретного этого датасета без изысков (стул будет называться стулом как ни крути), то можно соорудить целых два костыля (одинаковые концептуально): 
* искать каждое слово из названия в тексте ревью, потом фильтровать по каким-нибудь простым критерием, типа совпало больше двух слов, значит это entity
* вытащить из nltk'шного wordnet все слова, которые в дефиниции содержат слово мебель, и искать их по тексту (но это может быть с одной стороны слишком экстра (их будет сильно больше), с другой стороны точность может быть СИЛЬНО ниже, потому что в наш условных список entities на выходе может попасть что-то лишнее)

3. быстрый NLP (CountVectorizer в любой непонятной ситуации) -> группируем отзывы по товарам и ищем наиболее частотные слова для каждого предмета мебели, в надежде, что можно будет вычленить штуки, которые уникальны для конкретного предмета мебели, и которые еще и будут существительными при этом. Огромный плюс -- можно написать на коленке и закрытыми глазами (чем я сейчас и занимаюсь), из минусов -- все еще костыль? но это уже особенность датасета -- для названий мебели нужен не ner, а обычный entity-detection, потому что black chair все еще просто black chair, а не SuperBlackChair 10




In [4]:
def tokenize(sentence):
    return [word for word in word_tokenize(sentence.lower().translate(str.maketrans('', '', string.punctuation))) if word not in stopwords]

In [5]:
def get_entities(reviews, filter_pos):
    entities = []
    vectorizer = CountVectorizer(tokenizer=tokenize)
    mdl = vectorizer.fit_transform(reviews)
    vocab = vectorizer.get_feature_names_out()

    for i in range(mdl.shape[0]):
        most_popular = np.argpartition(mdl[i].toarray()[0], -1000)[-1000:]
        most_popular_with_tags = nltk.pos_tag(vocab[most_popular])
        out = [word for word, pos_tag in most_popular_with_tags if filter_pos in pos_tag]
        entities.extend(out)
    return set(entities)


In [6]:
entities = get_entities(df["review/text"].values.tolist(), 'NN')

In [7]:
def get_results(reviews, entities, search_quantity, method):
    tokens = [product for furniture in reviews for product in furniture]
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    search = nltk.collocations.BigramCollocationFinder.from_words(tokens)
    search.apply_word_filter(lambda x: x not in entities)
    if method == 'likelihood_ratio':
        return search.nbest(bigram_measures.likelihood_ratio, search_quantity)
    elif method == 'pmi':
        return search.nbest(bigram_measures.pmi, search_quantity)
    elif method == 'student':
        return search.nbest(bigram_measures.student_t, search_quantity)


In [8]:
reviews = df["review/text"].apply(tokenize).tolist()

In [9]:
get_results(reviews, entities, 13, 'likelihood_ratio')

[('put', 'together'),
 ('tv', 'stand'),
 ('coat', 'belt'),
 ('comfy', 'sturdywhen'),
 ('stoolsbuy', 'comfy'),
 ('sauder', 'cabinet'),
 ('bar', 'stoolsbuy'),
 ('check', 'dimensions'),
 ('dark', 'walnutbefore'),
 ('downyou', 'sit'),
 ('pocket', 'stuff'),
 ('stuff', 'coat'),
 ('video', 'tapes')]

In [10]:
get_results(reviews, entities, 13, 'pmi')

[('advice', 'reviewers'),
 ('alas', 'cheaper'),
 ('anchors', 'sheet'),
 ('anothera', 'google'),
 ('assemblei', 'hadnt'),
 ('avoid', 'guys'),
 ('bolts', 'add'),
 ('bumps', 'arm'),
 ('cause', 'complain'),
 ('cheaper', 'brickandmortar'),
 ('cherrywood', 'lends'),
 ('child', 'watches'),
 ('claim', 'theyve')]

In [11]:
get_results(reviews, entities, 13, 'student')

[('put', 'together'),
 ('tv', 'stand'),
 ('sauder', 'cabinet'),
 ('coffee', 'table'),
 ('media', 'cabinet'),
 ('piece', 'furniture'),
 ('product', 'bed'),
 ('buy', 'product'),
 ('coat', 'belt'),
 ('comfy', 'sturdywhen'),
 ('stoolsbuy', 'comfy'),
 ('bar', 'stoolsbuy'),
 ('check', 'dimensions')]

Вообще все плохо сработали, но pmi хуже всех, и мне стьюдент показался более правдоподобным

In [32]:
res = {}
for i, tokens in enumerate(reviews):
    for j, token in enumerate(tokens):
        if token in entities:
            res[token] = []
            if j > 0:
                res[token].append([tokens[j - 1], token])
            if j < len(tokens) - 2:
                res[token].append([token, tokens[j + 1]])
res

{'media': [['space', 'media'], ['media', 'cabinet']],
 'cabinet': [['large', 'cabinet'], ['cabinet', 'would']],
 'someone': [['product', 'someone'], ['someone', 'wants']],
 'assembly': [['frustration', 'assembly']],
 'roomy': [['cabinet', 'roomy'], ['roomy', 'enough']],
 'enough': [['stylish', 'enough'], ['enough', 'display']],
 'accommodate': [['enough', 'accommodate'], ['accommodate', 'many']],
 'vhs': [['games', 'vhs']],
 'videotapes': [['vhs', 'videotapes'], ['videotapes', 'cherrywood']],
 'cherrywood': [['videotapes', 'cherrywood'], ['cherrywood', 'lends']],
 'lends': [['cherrywood', 'lends'], ['lends', 'attractive']],
 'attractive': [['lends', 'attractive'], ['attractive', 'look']],
 'look': [['distressedantique', 'look'], ['look', 'top']],
 'room': [['domers', 'room']],
 'recommend': [['bedroom', 'recommend'], ['recommend', 'wood']],
 'delivery': [['quick', 'delivery'], ['delivery', 'plan']],
 'excellent': [['cabinet', 'excellent']],
 'merchandise': [['excellent', 'merchandise']